In [ ]:
from analysis.paper import *

In [ ]:
ags = load()

In [ ]:
ags

In [ ]:
plot_frontiers(ags)

In [ ]:
df = data.modelled_elos(ags)

In [ ]:
from cycler import cycler

In [ ]:
cycler?

In [ ]:
import matplotlib as mpl